<a href="https://colab.research.google.com/github/satishgc227/git-test/blob/master/Real_worked_out_otto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys

In [61]:
df=pd.read_csv("/content/traian csv.csv")

In [62]:
df.dropna()

,session,aid,ts,type
0,8000000,766953,1.660420e+12,clicks
1,8000000,1586093,1.660420e+12,clicks
2,8000000,1586093,1.660420e+12,clicks
3,8000000,682970,1.660420e+12,clicks
4,8000000,799794,1.660420e+12,clicks
...,...,...,...,...
1048570,8089703,911487,1.660460e+12,clicks
1048571,8089703,911487,1.660460e+12,carts
1048572,8089703,413885,1.660460e+12,clicks
1048573,8089703,911487,1.660460e+12,clicks


In [63]:
df.replace({'clicks':0,'carts':1,'orders':2},inplace=True)

In [64]:
df.dropna()

,session,aid,ts,type
0,8000000,766953,1.660420e+12,0
1,8000000,1586093,1.660420e+12,0
2,8000000,1586093,1.660420e+12,0
3,8000000,682970,1.660420e+12,0
4,8000000,799794,1.660420e+12,0
...,...,...,...,...
1048570,8089703,911487,1.660460e+12,0
1048571,8089703,911487,1.660460e+12,1
1048572,8089703,413885,1.660460e+12,0
1048573,8089703,911487,1.660460e+12,0


In [65]:
df=df[['session','type','aid']]

In [66]:
users = df.session.unique()
movies = df.type.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [67]:
df['session'] = df['session'].apply(lambda x: userid2idx[x])
df['type'] = df['type'].apply(lambda x: movieid2idx[x])
split = np.random.rand(len(df)) < 0.8
train = df[split]
valid = df[~split]
print(train.shape , valid.shape)

(839080, 3) (209495, 3)


In [68]:
#Creating an embedding layer for session
import keras
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model


In [69]:
n_session=len(df['session'].unique())
n_type=len(df['type'].unique())

n_latent_factors=10

In [70]:
session_input=Input(shape=(1,),name='session_input')
session_embedding=Embedding(n_session,n_latent_factors,name='session_embedding')(session_input)
session_vec=Flatten(name='session_flatten')(session_embedding)

In [71]:
type_input=Input(shape=(1,),name='type_input')
type_embedding=Embedding(n_type,n_latent_factors,name='type_embedding')(type_input)
type_vec=Flatten(name='type_flatten')(type_embedding)

In [72]:
from keras.layers import dot

In [73]:
sim=dot([session_vec,type_vec],name='Simalarity-Dot-Product',axes=1)
from keras.layers import Dropout, Flatten,Activation,Input,Embedding

In [74]:
model =keras.models.Model([session_input, type_input],sim)

In [85]:
model.compile(optimizer=Adam(lr=1e-4),loss='mse')

In [76]:
train.shape
batch_size=10000
epochs=10

In [77]:
History = nn_model.fit([train.session,train.type],train.aid, batch_size=batch_size,
                              epochs =epochs, validation_data = ([valid.session,valid.type],valid.aid),
                              verbose = 1)

Epoch 1/10
84/84 [==============================] - 3s 35ms/step - loss: 283368521728.0000 - val_loss: 276259373056.0000
Epoch 2/10
84/84 [==============================] - 3s 34ms/step - loss: 280177410048.0000 - val_loss: 276054900736.0000
Epoch 3/10
84/84 [==============================] - 3s 35ms/step - loss: 277349924864.0000 - val_loss: 275961970688.0000
Epoch 4/10
84/84 [==============================] - 4s 42ms/step - loss: 275122585600.0000 - val_loss: 275951157248.0000
Epoch 5/10
84/84 [==============================] - 5s 63ms/step - loss: 272922902528.0000 - val_loss: 276000473088.0000
Epoch 6/10
84/84 [==============================] - 6s 69ms/step - loss: 270669594624.0000 - val_loss: 276107722752.0000
Epoch 7/10
84/84 [==============================] - 5s 65ms/step - loss: 269107298304.0000 - val_loss: 276267433984.0000
Epoch 8/10
84/84 [==============================] - 4s 53ms/step - loss: 267354521600.0000 - val_loss: 276460863488.0000
Epoch 9/10
84/84 [==============

In [78]:
nn_inp=Dense(96,activation='relu')(sim)
nn_inp=Dropout(0.4)(nn_inp)
# nn_inp=BatchNormalization()(nn_inp)
nn_inp=Dense(1,activation='relu')(nn_inp)
nn_model =keras.models.Model([session_input, type_input],nn_inp)
nn_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 session_input (InputLayer)     [(None, 1)]          0           []                               
                                                                                                  
 type_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 session_embedding (Embedding)  (None, 1, 10)        897040      ['session_input[0][0]']          
                                                                                                  
 type_embedding (Embedding)     (None, 1, 10)        30          ['type_input[0][0]']             
                                                                                            

In [79]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

In [80]:
nn_model.compile(optimizer=Adam(lr=1e-3),loss='mse')

In [81]:
batch_size=10000
epochs=10

In [82]:
History = nn_model.fit([train.session,train.type],train.aid, batch_size=batch_size,
                              epochs =epochs, validation_data = ([valid.session,valid.type],valid.aid),
                              verbose = 1)

Epoch 1/20
84/84 [==============================] - 4s 36ms/step - loss: 1153037369344.0000 - val_loss: 1155439788032.0000
Epoch 2/20
84/84 [==============================] - 3s 34ms/step - loss: 1153032781824.0000 - val_loss: 1155431792640.0000
Epoch 3/20
84/84 [==============================] - 3s 35ms/step - loss: 1153016528896.0000 - val_loss: 1155404529664.0000
Epoch 4/20
84/84 [==============================] - 3s 34ms/step - loss: 1152969342976.0000 - val_loss: 1155333095424.0000
Epoch 5/20
84/84 [==============================] - 3s 35ms/step - loss: 1152858324992.0000 - val_loss: 1155179347968.0000
Epoch 6/20
84/84 [==============================] - 3s 35ms/step - loss: 1152639172608.0000 - val_loss: 1154895446016.0000
Epoch 7/20
84/84 [==============================] - 3s 35ms/step - loss: 1152257622016.0000 - val_loss: 1154423717888.0000
Epoch 8/20
84/84 [==============================] - 3s 34ms/step - loss: 1151651020800.0000 - val_loss: 1153700986880.0000
Epoch 9/20
84/84

In [83]:
nn_model.predict()

,session,type,aid
0,0,0,766953
1,0,0,1586093
2,0,0,1586093
3,0,0,682970
4,0,0,799794
...,...,...,...
1048570,89703,0,911487
1048571,89703,1,911487
1048572,89703,0,413885
1048573,89703,0,911487
